

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------
This notebook maps the matches into the twitter posts dataframe.
The matches schedule and results were obtained from https://www.skysports.com/manchester-united-results/2022-23
Additionally, it creates two datetime columns to indicate when the match started and ended, with a time window before and after, in order to capture the tweets that occured during the match.
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [7]:
import pandas as pd

In [8]:
matches=pd.read_csv("../data/raw/matches_May_20Oct.csv")
posts=pd.read_csv("../data/processed/Post_Twitter_all_Hashtags.csv")

In [9]:
# Import matches csv, add the dayfirst to make sure the day is not swapped with month
matches['match_datetime']=pd.to_datetime(matches['match_datetime'],dayfirst=True)

In [10]:
matches.groupby(['match_type']).size()

match_type
Friendly Match                 6
Premier League                13
UEFA Europa League Group E     4
dtype: int64

In [11]:
#Add the window during which the users could be tweeting: 1 hour before the match and 2hours after

matches['match_adj_start_time']=matches['match_datetime'].apply(lambda x: x+pd.DateOffset(minutes=-60))
matches['match_adj_end_time'] =matches['match_datetime'].apply(lambda x: x+pd.DateOffset(hours=3))
posts['Date_Created']=pd.to_datetime(posts['Date_Created'])
posts['Date_Created_UTC'] = posts['Date_Created'].dt.tz_convert(None)

In [6]:
#map the match number into the tweets dataframe, if no match use 999
posts['match']=999
for row, index in matches.iterrows():
    posts.loc[posts['match']==999,'match'] =posts.loc[posts['match']==999,'Date_Created_UTC'].apply(lambda x: row if ((x >= matches.loc[row,'match_adj_start_time'])  & (x <= matches.loc[row,'match_adj_end_time'])) else 999)

In [7]:
# Merge the post df with the matches df based on match number
posts=pd.merge(posts,matches,how='left',left_on='match',right_index=True)

In [8]:
posts.to_csv('../data/processed/Post_Twitter_all_Hashtags_with_matches.csv', encoding='utf-8', index = False)